# Полезные мелочи

## Функции с `*args`

Обычно мы пишем функцию и задаем в ней определенное количество аргументов. Такая функция упадет, если мы дадим ей больше аргументов, чем она ждет.

In [1]:
def f(x, y, z):
    return x, y, z

print(f(1, 2, 3))
print(f(1, 2, 3, 4, 5))

(1, 2, 3)


TypeError: f() takes 3 positional arguments but 5 were given

Иногда нужно написать функцию, которая может принять любое количество аргументов, но мы не знаем сколько их. Для этого используется специальный синтаксис:

In [2]:
def f_with_arguments(x, *args):
    return x, args
print(f_with_arguments(1))
print(f_with_arguments(1, 2, 3, 4))
print(f_with_arguments(1, 2, 3, 4, 100, 3000))

(1, ())
(1, (2, 3, 4))
(1, (2, 3, 4, 100, 3000))


In [5]:
def f2(*args):
    return args
print(f2())
print(f2(1))
print(f2(1,2,3,4))

()
(1,)
(1, 2, 3, 4)


Функции с опциональными аргументами (`*args`) позволяют писать более чистый код и уменьшают визуальный шум.

Например, часто вам понадобится использовать функцию, которая записывает логи. Предположим, мы получаем сообщение и набор значений, и хотим записывать их в лог.

In [24]:
import time
def log(message, values):
    print(time.strftime("[%H:%M:%S]"), 'Сообщение:', message, ', '.join(str(i) for i in values))
log('Новые числа', [1, 2])  # передаем массив
log('Привет!', [])  # передаем пустой массив

[03:05:50] Сообщение: Новые числа 1, 2
[03:05:50] Сообщение: Привет! 


Когда у нас нет значений, приходится передавать функции пустой массив. Лучше все-таки совсем не писать второй аргумент. Использование `*args` как раз позволяет это сделать. 

In [25]:
import time
def log(message, *values):
    print(time.strftime("[%H:%M:%S]"), 'Сообщение:', message, ', '.join(str(i) for i in values))
log('Новые числа', 1, 2)
log('Привет!')

[03:05:57] Сообщение: Новые числа 1, 2
[03:05:57] Сообщение: Привет! 


Если у нас уже есть список и мы хотим передать его в нашу функцию `log`, мы снова можем использовать оператор `*`. Таким образом мы говорим питону передать функции не массив (как один объект), а элементы массива как позиционные аргументы.

In [26]:
data = [100, 3000, 27, 7787]
log('Новые данные', *data)

[03:06:02] Сообщение: Новые данные 100, 3000, 27, 7787


### Проблемы, которые могут быть у функций с неизвестным числом аргументов:

1) Все дополнительные аргументы сначала добавляются в кортеж, и только потом передаются в нашу функцию. Так что если мы передаем в нашу функцию значения, возвращаемые генератором, то функция не запустится, пока генератор не закончит работать. В итоге наша программа может съесть много памяти и вообще может упасть. Поэтому функции с `*args` разумно использовать, когда мы не знаем количество аргументов, но при этом знаем, что это количество не супер-большое.

In [21]:
def my_generator():
    for i in range(10):
        yield i
        
def my_func(*args):
    print(args)
    
it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


2) Если функция использует `*args` и при этом используется в каком-то коде, то мы не можем добавить в эту функцию новые позиционные аргументы. То есть можем, но для этого придется отредактировать кажду строчку, где эта функция вызывается.

In [27]:
def log(new_arg, message, *values):
    print(time.strftime("[%H:%M:%S]"), new_arg, 'Сообщение:', message, ', '.join(str(i) for i in values))
        
log(1, 'Новые числа', 7, 33) # Отредактировали функцию и вызываем по-новому - все окей
log('Новые числа', 1, 2)    # используем старый вызов - все срабатывает немножко не так

[03:06:10] 1 Сообщение: Новые числа 7, 33
[03:06:10] Новые числа Сообщение: 1 2


Так появляются баги, которые сложно чинить: программа не падает с ошибкой, но начинает работать неожиданным образом. Чтобы этого избежать, часто пользуются функциями с keyword arguments.

## Функции с именованными аргументами 

Позиционные аргументы всегда можно передавать по названию аргумента (keyword arguments):

In [29]:
def remainder(number, divisor):
    return number % divisor

print(remainder(20, 7))  # передаем аргументы в том же порядке, в котором они перечислены в определении функции
print(remainder(20, divisor=7))  # можно передать некоторые аргументы по имени
print(remainder(number=20, divisor=7))  # можно передать все аргументы по имени
print(remainder(divisor=7, number=20))  # при этом не важно, в каком порядке

6
6
6
6


Позиционные аргументы ВСЕГДА нужно передавать до keyword-аргументов, иначе все упадет.

In [30]:
remainder(number=20, 7)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-30-fa871e527313>, line 1)

Каждый аргумент нужно передать только один раз.

In [31]:
remainder(20, number=7)

TypeError: remainder() got multiple values for argument 'number'

### Плюсы функций с keyword arguments:
1) гибкость (передаем в любом порядке)

2) ясность (например, в `remainder(20, 7)` не ясно, что делимое, а что делитель, а в `remainder(divisor=7, number=20)` все очевидно)

3) каждому аргументу можно задать значение по умолчанию, которое будет использоваться чаще всего (!меньше повторяющегося кода!)

Например, мы пишем функцию, которая делит предложения на слова. Этой функции можно задать какое-то поведение по умолчанию, которое можно менять с помощью именованных аргументов.

In [32]:
def split_to_words(sent, strip_punct=True, lower=True, punct=',.!?;:'):
    if lower:
        sent = sent.lower()
    words = sent.split()
    if strip_punct:
        words = [i.strip(punct) for i in words]
    return words

print(split_to_words('Привет, мир!'))
print(split_to_words('Привет, мир!', strip_punct=False, lower=False))

['привет', 'мир']
['Привет,', 'мир!']


Когда мы используем в функции значения по умолчанию, которые могут изменяться, лучше устанавливать в качестве их первого значения None. 
Примеры.
1) массивы:

In [33]:
def func(x, arr=[]):
    arr.append(x)
    return arr

print(func('Первый вызов функции'))
print(func('Второй вызов'))  # добавляется в первый массив
k = func('Вызов с массивом', arr=[]) # создается новый массив
print(k) 
print(func('Сюрприз!')) # добавляется в первый массив

['Первый вызов функции']
['Первый вызов функции', 'Второй вызов']
['Вызов с массивом']
['Первый вызов функции', 'Второй вызов', 'Сюрприз!']


Чтобы функция вела себя более предсказуемо, используем None.

In [36]:
def func(x, arr=None):
    if arr is None:
        arr = []
    arr.append(x)
    return arr

print(func('Первый вызов функции')) # создается новый массив
print(func('Второй вызов'))  # создается новый массив 
k = func('Вызов с массивом', arr=[]) # создается новый массив
print(k) 
print(func('Сюрприз!', arr=k)) # добавляется в предыдущий массив

['Первый вызов функции']
['Второй вызов']
['Вызов с массивом']
['Вызов с массивом', 'Сюрприз!']


2) метка времени

In [41]:
def log(message, when=time.strftime("[%H:%M:%S]")):
    print(when, message)
    
log('Привет!')
time.sleep(3)
log('Привет еще раз!')

[03:41:04] Привет!
[03:41:04] Привет еще раз!


Временная метка не поменялась, потому что функция `time.strftime("[%H:%M:%S]")` запускается только при первом вызове функции, а не при каждом вызове!

То же самое касается словарей, множеств и вообще любых типов, значение которых может изменяться. 

## Функции с `**kwargs`

Так же как мы можем передвать функции любое количество аргументов с помощью `*args`, мы можем передавать ей любые именованные аргументы с помощью `**kwargs`. `**kwargs` представляется в питоне как словарь.

In [42]:
def func(**kwargs):
    return kwargs
print(func(a=1, b=2))

{'a': 1, 'b': 2}


Можно использовать `*args` и `**kwargs` одновременно.

In [43]:
def cool_func(*args, **kwargs):
    return args, kwargs
print(cool_func(1, 2, x=1, y=3))

((1, 2), {'y': 3, 'x': 1})


И точно так же, как со списками, мы можем передавать в функцию готовые словари в качестве `**kwargs`.

In [44]:
d = {'roses': 'red', 'violets': 'blue', 'grass': 'green'}
print(cool_func(**d))

((), {'roses': 'red', 'grass': 'green', 'violets': 'blue'})


Очень удобно использовать `**kwargs` при создании объектов. Почему? 

Атрибуты экземпляров объектов и их значения в питоне хранятся в словаре. Ко всем объектам можно применять функцию `vars()`, которая позволяет посмотреть этот словарь. А так же этот словарь можно менять на лету.

In [46]:
class Object:
    def __init__(self):
        pass


def create_person(name, surname, age, status):
    result = Object()
    result.name = name
    result.surname = surname
    result.age = age
    result.status = status
    return result

x = create_person('Linus', 'Torvalds', 48, 'Software engineer')
print(vars(x))

{'name': 'Linus', 'age': 48, 'surname': 'Torvalds', 'status': 'Software engineer'}


In [47]:
vars(x)['birthday'] = 'December 28'
print(x.birthday)

December 28


In [48]:
print(vars(x))

{'name': 'Linus', 'age': 48, 'surname': 'Torvalds', 'birthday': 'December 28', 'status': 'Software engineer'}


Эти знания мы можем применять, чтобы легко и просто создавать объекты. Например так:

In [50]:
class FurryAnimal:
    pass

def create(**values):
    result = FurryAnimal()
    vars(result).update(values)
    return result

x = create(species='Cat', furriness=100500, color='rainbow')
print(x.species)
print(x.color)

Cat
rainbow


Или даже еще проще:

In [51]:
class Struct:
    def __init__(self, **values):
        vars(self).update(values)
        
x = Struct(species='Nyan-Cat', furriness=100500, color='rainbow')
print(x.species, x.color)

Nyan-Cat rainbow


# Задания

Разберите какой-нибудь текст с помощью Mystem командой

    mystem <input >output -cnisd --eng-gr --format json
    
(Или возьмите готовый файл - `python_mystem.json` в этом репозитории.)

Создайте класс Word, в котором будут храниться словоформа, количество разборов, самая частая лемма данного слова, самая частая часть речи этого слова. 

Нужно создать массив экземпляров класса Word, при этом для каждой словоформы в массив должен быть добавлен только один экземпляр.

В вашей программе должна быть хотя бы одна функция (или метод) с `*args` и хотя бы одна функция (или метод) с `**kwargs`.